# Домашнее задание №2

Асланов А.Б., ИУ9-21М

## Задание 2

Запросы - это проанализированные факты из Википедии.    
Документы - это предложения из статей Википедии, указанных в этих фактах.   

Обработать морфологическим анализатором и найти наиболее релевантные предложения:
- По векторной модели без `idf`;
- По TF-IDF (`df` в данном случае - количество предложений, в которых встретилось слово).    

Нормировать запросы и предложения, то есть выстроить все предложения из статей по мере сходства с запросом по векторной модели.    
В отчёте должны быть показаны веса выдаваемых предложений.

----------------------------------------------------------------------------------------

**Запросы** (== факты):

1) В медленном движении медленно читают;    
2) Петру I доставляли из Китая посылки с золотым песком;    
3) Зоны отдыха экипажей дальнемагистральных авиарейсов находятся над или под пассажирским салоном.    

**Документы**:

*К факту 1:*    
Д1.1: Медленное чтение как часть медленного движения;    
Д1.2: Однако в последнее время наблюдается повышенный интерес к медленному чтению как направлению в рамках медленного движения.

*К факту 2 прямо относящихся предложений из статьи не было. Наиболее подходящие косвенные:*    
Д2.1: Коробчатое золото — золотой песок, который поставлялся в Российскую империю из Китая для чеканки червонцев.    
Д2.2: Коробчатое золото использовалось для чеканки червонцев в 1701 и 1704 годах.

*К факту 3:*    
Д3.1: Отсеки для отдыха экипажа обычно присутствуют на дальнемагистральных лайнерах и могут располагаться над пассажирским салоном (в этом случае в них нужно подниматься по лестнице) или рядом с ним.

In [1]:
from pymystem3 import Mystem
m = Mystem()

from collections import Counter
import re
import string

import pandas as pd
import numpy as np

In [2]:
def make_morph_analysis(fact):
    """Проводим морфологический анализ для последующей обработки"""
    
    # Удлить все знаки препинания
    fact = ''.join(ch for ch in fact if ch not in "[«»;,()'`-—.?!:']")
    
    # Сделать все слова предложения в нижнем регистре
    fact = fact.lower()
    
    # Лемматизация
    fact = m.lemmatize(fact)
    fact = ''.join(fact)
    return fact

In [3]:
docs = ['Отсеки для отдыха экипажа обычно присутствуют на дальнемагистральных лайнерах и могут располагаться над пассажирским салоном (в этом случае в них нужно подниматься по лестнице) или рядом с ним',
        'Зона отдыха экипажа — отсек или специально отведённое место внутри самолёта, предназначенное для отдыха членов экипажа при выполнении дальних рейсов.',
        'Члены экипажа могут воспользоваться зонами отдыха в нерабочее время',
        'Модуль либо имеет форму стандартного грузового контейнера и помещается в грузовой отсек самолёта, расположенный под пассажирским салоном (вход в него осуществляется через специальный люк, скрытый от пассажиров), либо предназначен для установки внутри пассажирского салона', 
        'Так, на дальнемагистральных самолётах Аэрофлота обособленный отсек отдыха есть только у лётного экипажа, бортпроводники могут отдыхать только на задних рядах салонов, если там нет пассажиров', 
        'Отсеки отдыха также могут присутствовать на транспортных и военно-транспортных самолётах',
        'Секции для пилотов, расположенные в передней части самолёта, отделены от расположенных сзади секций для бортпроводников.',
        'Исключения могут быть сделаны для отсеков, оборудованных креслами с ремнями']

query = ['Зоны отдыха экипажей дальнемагистральных авиарейсов находятся над или под пассажирским салоном']






In [4]:
def extract_all_docs_words(docs):
    """Извлекаем все слова из документа или запроса и прогоняем их через морфологический анализатор"""
    
    all_docs_words = []
    for doc in docs:
        all_docs_words += doc.split()
    all_docs_words = ' '.join(all_docs_words)
    all_docs_words = make_morph_analysis(all_docs_words)
    all_docs_words = list(set(all_docs_words.split()))
    return all_docs_words

all_docs_words = extract_all_docs_words(docs)

In [5]:
def check_the_same_words(query, doc, counter):
    """Находим количество совпадений слов в запросе и документе"""
    
    # Морфологическая обработка поданного на вход документа
    doc = make_morph_analysis(doc)
    
    # Разбиваем предложение формата <str> на список
    doc = doc.split()

    # Словари с частотами слов в запросе и в документе
    d_doc = Counter(doc)
    
    # Если i-ое слово из запроса есть в документе, то считаем частотность этого i-го слова в документе.
    # Если же такого слова нет, ставим 0.
    doc_vec = []
    for word in all_docs_words:
        if word in doc:
            doc_vec.append(d_doc[word])
        else:
            doc_vec.append(0)
    
    d = {f'd_{counter}':doc_vec}
    dataframe = pd.DataFrame(data=d, index=all_docs_words)
    return dataframe
            
            
def make_termdoc_matrix(query, docs):
    """Создаём матрицы терм-документ"""
    
    counter = 0 # счётчик документов
    dataframe = pd.DataFrame()
    for doc in docs:
        counter += 1
        dataframe_curr = check_the_same_words(query, doc, counter)
        dataframe = pd.concat([dataframe, dataframe_curr], axis=1)

    # Считаем df как кол-во предложений, в которых встретилось i-ое слово запроса.
    # Его проще вычислить по датафрейму: df будет равен количеству ненулевых столбцов для i-го слова.
    dataframe['df'] = pd.Series((dataframe != 0).astype(int).sum(axis=1))
    
    
    # Добавляем в датафрейм столбец с запросом
    query = make_morph_analysis(query)
    query = query.split()
    d_query = Counter(query)
    query_vec = []
    for word in all_docs_words:
        if word in query:
            query_vec.append(d_query[word])
        else:
            query_vec.append(0)
    
    dataframe['q'] = query_vec
    
    return dataframe


dataframe = make_termdoc_matrix(query, docs)
dataframe

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,df,q
располагать,0,0,0,1,0,0,2,0,2,0
ряд,0,0,0,0,1,0,0,0,1,0
при,0,1,0,0,0,0,0,0,1,0
оно,0,0,0,1,0,0,0,0,1,0
иметь,0,0,0,1,0,0,0,0,1,0
аэрофлот,0,0,0,0,1,0,0,0,1,0
там,0,0,0,0,1,0,0,0,1,0
люк,0,0,0,1,0,0,0,0,1,0
под,0,0,0,1,0,0,0,0,1,1
предназначать,0,1,0,1,0,0,0,0,2,0


### 1. Векторная модель (без учета `idf`)

Вес определяется как функция от количества вхождений терма в документ.




In [6]:
import operator
from scipy.spatial import distance


def calculate_cosine_similarity(dataframe, doc_num):
    """Расчёт косинусного расстояния для векторной модели"""
    
    document = np.array(dataframe[f'd_{doc_num}'])
    query = np.array(dataframe['q'])
    return 1 - distance.cosine(document, query)

# Запись в словарь всех косинусных расстояний для каждого конкретного документа
answers = {}
for i, doc_num in zip(range(len(docs)), range(1, len(docs)+1)):
    answers[docs[i]] = calculate_cosine_similarity(dataframe, doc_num)
    
# Сортировка по значениям словаря для упорядочения порядка ранжирования

def sort_answers(answers, query):
    print('Запрос: ', query, '\n'*3)
    for ans in sorted(answers, key=answers.get, reverse=True):
        print(ans, answers[ans])
        
final_answers = sort_answers(answers, query)

Запрос:  ['Зоны отдыха экипажей дальнемагистральных авиарейсов находятся над или под пассажирским салоном'] 



Отсеки для отдыха экипажа обычно присутствуют на дальнемагистральных лайнерах и могут располагаться над пассажирским салоном (в этом случае в них нужно подниматься по лестнице) или рядом с ним 0.42600643361512924
Зона отдыха экипажа — отсек или специально отведённое место внутри самолёта, предназначенное для отдыха членов экипажа при выполнении дальних рейсов. 0.41702882811414954
Члены экипажа могут воспользоваться зонами отдыха в нерабочее время 0.33333333333333326
Модуль либо имеет форму стандартного грузового контейнера и помещается в грузовой отсек самолёта, расположенный под пассажирским салоном (вход в него осуществляется через специальный люк, скрытый от пассажиров), либо предназначен для установки внутри пассажирского салона 0.25125945381480297
Так, на дальнемагистральных самолётах Аэрофлота обособленный отсек отдыха есть только у лётного экипажа, бортпроводники могут

### 2. Модель $TF-IDF$

Вес определяется как произведение функции от количества вхождений терма в документ и функции от величины, обратной количеству документов коллекции, в которых встречается этот терм.



### Расчёт косинусного расстояния для векторной модели
  
Косинусное расстояние позволяет вычислить меру сходства двух документов как $cos(\vec{q},\vec{d}) = \frac{\vec{q}\vec{d}}{||\vec{q}||*||\vec{d}||}$, где $q$ - вектор запроса, а $d$ - вектор документа. 

In [11]:
from scipy.spatial import distance
import math

def calculate_cosine_similarity_tfidf(dataframe, doc_num, N=len(docs)):
    
    tf = np.array(dataframe[f'd_{doc_num}'])
    df = np.array(dataframe['df'])
    idf = np.log10(N / df)
    
    document = tf * idf
    query = np.array(dataframe['q'])
    return 1 - distance.cosine(document, query)


# Запись в словарь всех косинусных расстояний для каждого конкретного документа
answers_tfidf = {}
for i, doc_num in zip(range(len(docs)), range(1, len(docs)+1)):
    answers_tfidf[docs[i]] = calculate_cosine_similarity_tfidf(dataframe, doc_num)

sort_answers(answers_tfidf, query)

Запрос:  ['Зоны отдыха экипажей дальнемагистральных авиарейсов находятся над или под пассажирским салоном'] 



Отсеки для отдыха экипажа обычно присутствуют на дальнемагистральных лайнерах и могут располагаться над пассажирским салоном (в этом случае в них нужно подниматься по лестнице) или рядом с ним 0.32961809135077114
Зона отдыха экипажа — отсек или специально отведённое место внутри самолёта, предназначенное для отдыха членов экипажа при выполнении дальних рейсов. 0.258567762497498
Модуль либо имеет форму стандартного грузового контейнера и помещается в грузовой отсек самолёта, расположенный под пассажирским салоном (вход в него осуществляется через специальный люк, скрытый от пассажиров), либо предназначен для установки внутри пассажирского салона 0.2039570167126472
Члены экипажа могут воспользоваться зонами отдыха в нерабочее время 0.19909818392541212
Так, на дальнемагистральных самолётах Аэрофлота обособленный отсек отдыха есть только у лётного экипажа, бортпроводники могут от

## Результаты

Жирным шрифтом выделены предложения, которые, как ожидал при выделении вручную, должны быть наиболее релевантны запросу. Однако сейчас соглашусь с моделями, которые утверждают, что предложение *Коробчатое золото использовалось для чеканки червонцев в 1701 и 1704 годах* не является настолько подходящим, как предложения, которым присвоены бОльшие вероятности. 

В целом системы на данных примерах сопоставимы по качеству, однако TF-IDF присваивает вариантам в целом меньшие веса, то есть более "не уверен" в верности выдачи.

<img src="res1.png" width=950, height=950>
<img src="res2.png" width=950, height=950>
<img src="res4.png" width=950, height=950>

# Перепроверка

Возьмём результаты векторной модели и посмотрим, правильно ли было вычислено косинусное расстояние между векторами слов.

Здесь запрос $q$: "В медленном движении медленно читают", а документ $d$: "Прежде всего, будем говорить медленно" (предложения лемматизированы).

In [8]:
from scipy.spatial import distance

q = [1, 1, 1, 1, 1, 0, 0, 0, 0]   # в медленный движение медленно читать
d = [0, 0, 0, 1, 0, 1, 1, 1, 1]   # прежде всего быть говорить медленно


print(1 - distance.cosine(d, q))   # б/м недоставание до 0,2 в этом случае - издержка интерпретатора python'а

0.19999999999999996


<img src="5.png" width=200, height=200>

$cos(\theta) = \frac{q*d}{||q||*||d||} = \frac{0+0+0+1+0+0+0+0+0}{\sqrt{1^2+1^2+1^2+1^2+1^2+0^2+0^2+0^2+0^2}\sqrt{0^2+0^2+0^2+1^2+0^2+1^2+1^2+1^2+1^2}} = \frac{1}{\sqrt{5}\sqrt{5}} = 0,2$

# Перепроверка 2

Проверка вычисления векторной модели между запросом $q = $**В медленном движении медленно читают** и документом $d=$**Медленное движение как часть медленного движения**.

<img src="6.png" width=200, height=200>

$cos(\theta) = \frac{\vec{q}*\vec{d}}{||\vec{q}||*||\vec{d}||} = \frac{0+2+1+0+0+0+0+0}{\sqrt{0^2+2^2+1^2+0^2+0^2+1^2+1^2+1^2}\sqrt{1^2+1^2+1^2+1^2+1^2+0^2+0^2+0^2}} = \frac{3}{\sqrt{8}\sqrt{5}} \approx 0,474$